In [1]:
import pandas as pd

In [2]:
TARGET_COLUMNS = [ 
    'hospital_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'hcpcs_cpt',
    'modifiers',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_units',
    'billing_class',
    'setting',
    'payer_category',
    'payer',
    'plan',
    'payer',
    'plan',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_payer_notes'
]

In [3]:
ccn = "390328"
url = "https://dam.upmc.com/-/media/upmc/locations/hospitals/documents/cdm-json-files/274814831_upmc-east_standardcharges.csv"
filename = url.split("/")[-1]
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [4]:
import subprocess
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘274814831_upmc-east_standardcharges.csv’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://dam.upmc.com/-/media/upmc/locations/hospitals/documents/cdm-json-files/274814831_upmc-east_standardcharges.csv', '-O', '274814831_upmc-east_standardcharges.csv'], returncode=1)

In [5]:
df_in = pd.read_csv(filename, dtype='str', encoding='latin-1', low_memory=False)
df_in

,Hospital,Description,CPT/DRG,NDC,IP Price,OP Price,IP Cash Price,OP Cash Price,Min Rate,Max Rate,...,PA Health & Wellness MA,United Community Plan,Amerihealth,Geisinger Family,UPMC Comm,BC Comm,Aetna Comm,United Comm,Cigna Comm,Intergroup
0,UPMC East,FNA BX W/US GDN 1ST LES,10005,NaN,$798.00,$798.00,$638.40,$638.40,$0.00,"$1,198.07",...,"$1,198.07",$0.00,$995.22,$91.93,$926.67,$446.88,$319.20,$313.69,$311.22,$0.00
1,UPMC East,FNA BX W/MR GDN EA ADDL,10012,NaN,$798.00,$798.00,$638.40,$638.40,$0.00,$446.88,...,$202.73,$0.00,$168.41,$91.93,$391.42,$446.88,$319.20,$313.69,$311.22,NaN
2,UPMC East,FNA BIOPSY W/O IMA GDE 1ST LES,10021,NaN,"$1,370.00","$1,370.00","$1,096.00","$1,096.00",$65.35,"$1,198.07",...,"$1,198.07",$65.35,$995.22,$157.82,$94.50,$767.20,$548.00,$538.55,$534.30,NaN
3,UPMC East,IMG GUID DRN SOFT TIS CTH PERC,10030,NaN,"$1,654.00","$1,654.00","$1,323.20","$1,323.20",$131.54,$926.24,...,$193.42,$131.54,$160.67,$190.54,$882.69,$926.24,$661.60,$650.19,$645.06,NaN
4,UPMC East,ID SIMPLE ABSCESS,10060,NaN,$761.00,$761.00,$608.80,$608.80,$0.00,$955.67,...,$955.67,$649.95,$793.87,$87.67,$182.99,$426.16,$304.40,$299.15,$296.79,$0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7868,UPMC East,HIV w major related condition w MCC,974,NaN,"$236,460.50",NaN,"$189,168.40",NaN,"$15,735.07","$94,584.20",...,"$33,150.36","$26,601.16","$33,105.14",NaN,"$22,429.19","$42,271.99","$91,746.67","$91,604.80","$94,584.20",NaN
7869,UPMC East,Extensive O.R. procedure unrelated to principa...,981,NaN,"$155,903.50",NaN,"$124,722.80",NaN,"$7,435.15","$64,423.63",...,"$9,265.68","$7,435.15","$9,253.05",NaN,"$38,468.50","$64,423.63","$60,490.56","$60,397.02","$62,361.40",NaN
7870,UPMC East,Extensive O.R. procedure unrelated to principa...,982,NaN,"$93,262.98",NaN,"$74,610.38",NaN,"$9,984.40","$37,305.19",...,"$12,442.56","$9,984.40","$12,425.59",NaN,"$21,146.21","$32,168.04","$36,186.04","$36,130.08","$37,305.19",NaN
7871,UPMC East,Non-extensive O.R. proc unrelated to principal...,987,NaN,"$104,525.25",NaN,"$83,620.20",NaN,"$19,112.10","$43,918.04",...,NaN,NaN,NaN,NaN,"$27,309.34","$43,918.04","$40,555.80","$40,493.08","$41,810.10",NaN


In [6]:
df_mid = pd.DataFrame(df_in)
del df_mid['Hospital']
df_mid = df_mid.rename(columns={
    'Description': 'description',
    'CPT/DRG': 'code',
    'NDC': 'ndc',
})
df_mid

,description,code,ndc,IP Price,OP Price,IP Cash Price,OP Cash Price,Min Rate,Max Rate,UPMC For Life,...,PA Health & Wellness MA,United Community Plan,Amerihealth,Geisinger Family,UPMC Comm,BC Comm,Aetna Comm,United Comm,Cigna Comm,Intergroup
0,FNA BX W/US GDN 1ST LES,10005,NaN,$798.00,$798.00,$638.40,$638.40,$0.00,"$1,198.07",$572.05,...,"$1,198.07",$0.00,$995.22,$91.93,$926.67,$446.88,$319.20,$313.69,$311.22,$0.00
1,FNA BX W/MR GDN EA ADDL,10012,NaN,$798.00,$798.00,$638.40,$638.40,$0.00,$446.88,$0.00,...,$202.73,$0.00,$168.41,$91.93,$391.42,$446.88,$319.20,$313.69,$311.22,NaN
2,FNA BIOPSY W/O IMA GDE 1ST LES,10021,NaN,"$1,370.00","$1,370.00","$1,096.00","$1,096.00",$65.35,"$1,198.07",$317.74,...,"$1,198.07",$65.35,$995.22,$157.82,$94.50,$767.20,$548.00,$538.55,$534.30,NaN
3,IMG GUID DRN SOFT TIS CTH PERC,10030,NaN,"$1,654.00","$1,654.00","$1,323.20","$1,323.20",$131.54,$926.24,$572.05,...,$193.42,$131.54,$160.67,$190.54,$882.69,$926.24,$661.60,$650.19,$645.06,NaN
4,ID SIMPLE ABSCESS,10060,NaN,$761.00,$761.00,$608.80,$608.80,$0.00,$955.67,$165.08,...,$955.67,$649.95,$793.87,$87.67,$182.99,$426.16,$304.40,$299.15,$296.79,$0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7868,HIV w major related condition w MCC,974,NaN,"$236,460.50",NaN,"$189,168.40",NaN,"$15,735.07","$94,584.20","$15,842.01",...,"$33,150.36","$26,601.16","$33,105.14",NaN,"$22,429.19","$42,271.99","$91,746.67","$91,604.80","$94,584.20",NaN
7869,Extensive O.R. procedure unrelated to principa...,981,NaN,"$155,903.50",NaN,"$124,722.80",NaN,"$7,435.15","$64,423.63","$27,170.76",...,"$9,265.68","$7,435.15","$9,253.05",NaN,"$38,468.50","$64,423.63","$60,490.56","$60,397.02","$62,361.40",NaN
7870,Extensive O.R. procedure unrelated to principa...,982,NaN,"$93,262.98",NaN,"$74,610.38",NaN,"$9,984.40","$37,305.19","$14,935.82",...,"$12,442.56","$9,984.40","$12,425.59",NaN,"$21,146.21","$32,168.04","$36,186.04","$36,130.08","$37,305.19",NaN
7871,Non-extensive O.R. proc unrelated to principal...,987,NaN,"$104,525.25",NaN,"$83,620.20",NaN,"$19,112.10","$43,918.04","$19,288.91",...,NaN,NaN,NaN,NaN,"$27,309.34","$43,918.04","$40,555.80","$40,493.08","$41,810.10",NaN


In [7]:
money_columns = df_mid.columns.to_list()[3:]

def cleanup_dollar_value(value):
    if type(value) != str:
        return value
    
    if value == "Cost":
        return None
    
    if "N/A" in value:
        return None
    
    if type(value) == str:
        return value.replace(",", "").replace("$", "").replace("(", "").replace(")", "").strip()

    return value

def cleanup_values(values):
    return list(map(lambda value: cleanup_dollar_value(value), values))

df_mid[money_columns] = df_mid[money_columns].apply(lambda values: cleanup_values(values))

In [8]:
money_columns

['IP Price',
 'OP Price',
 'IP Cash Price',
 'OP Cash Price',
 'Min Rate',
 'Max Rate',
 'UPMC For Life',
 'Sec Blue',
 'United MC',
 'Aetna MC',
 'Aetna MC Advantra',
 'Gateway MC',
 'Humana',
 'PA Health & Wellness MC',
 'Community Life PACE',
 'Geisinger',
 'UPMC For You',
 'Gateway MA',
 'PA Health & Wellness MA',
 'United Community Plan',
 'Amerihealth',
 'Geisinger Family',
 'UPMC Comm',
 'BC Comm',
 'Aetna Comm',
 'United Comm',
 'Cigna Comm',
 'Intergroup']

In [9]:
remaining_columns = df_mid.columns.to_list()[:3] # XXX: for some files the cutoff has to be at index 2
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer', value_name='rate')
df_mid['payer'] = df_mid['payer'].str.strip()
df_mid

,description,code,ndc,payer,rate
0,FNA BX W/US GDN 1ST LES,10005,NaN,IP Price,798.00
1,FNA BX W/MR GDN EA ADDL,10012,NaN,IP Price,798.00
2,FNA BIOPSY W/O IMA GDE 1ST LES,10021,NaN,IP Price,1370.00
3,IMG GUID DRN SOFT TIS CTH PERC,10030,NaN,IP Price,1654.00
4,ID SIMPLE ABSCESS,10060,NaN,IP Price,761.00
...,...,...,...,...,...
220439,HIV w major related condition w MCC,974,NaN,Intergroup,NaN
220440,Extensive O.R. procedure unrelated to principa...,981,NaN,Intergroup,NaN
220441,Extensive O.R. procedure unrelated to principa...,982,NaN,Intergroup,NaN
220442,Non-extensive O.R. proc unrelated to principal...,987,NaN,Intergroup,NaN


In [10]:
def payer_category_from_payer_orig(payer_orig):
    if payer_orig == "IP Price" or payer_orig == "OP Price" or payer_orig == "Gross Charges IP" or payer_orig == "Gross Charges OP":
        return "gross"
    elif payer_orig == "IP Cash Price" or payer_orig == "OP Cash Price" or payer_orig == "Discounted Cash Price IP" or payer_orig == "Discounted Cash Price OP":
        return "cash"
    elif payer_orig == "Min Rate":
        return "min"
    elif payer_orig == "Max Rate":
        return "max"
    
    return "payer"

df_mid['payer_category'] = df_mid['payer'].apply(payer_category_from_payer_orig)
df_mid

,description,code,ndc,payer,rate,payer_category
0,FNA BX W/US GDN 1ST LES,10005,NaN,IP Price,798.00,gross
1,FNA BX W/MR GDN EA ADDL,10012,NaN,IP Price,798.00,gross
2,FNA BIOPSY W/O IMA GDE 1ST LES,10021,NaN,IP Price,1370.00,gross
3,IMG GUID DRN SOFT TIS CTH PERC,10030,NaN,IP Price,1654.00,gross
4,ID SIMPLE ABSCESS,10060,NaN,IP Price,761.00,gross
...,...,...,...,...,...,...
220439,HIV w major related condition w MCC,974,NaN,Intergroup,NaN,payer
220440,Extensive O.R. procedure unrelated to principa...,981,NaN,Intergroup,NaN,payer
220441,Extensive O.R. procedure unrelated to principa...,982,NaN,Intergroup,NaN,payer
220442,Non-extensive O.R. proc unrelated to principal...,987,NaN,Intergroup,NaN,payer


In [12]:
def patient_class_from_payer_orig(payer_orig):
    if "IP" in payer_orig:
        return "inpatient"
    elif "OP" in payer_orig:
        return "outpatient"
    return ""

df_mid['setting'] = df_mid['payer'].apply(patient_class_from_payer_orig)
df_mid                                     

,description,code,ndc,payer,rate,payer_category,setting
0,FNA BX W/US GDN 1ST LES,10005,NaN,IP Price,798.00,gross,inpatient
1,FNA BX W/MR GDN EA ADDL,10012,NaN,IP Price,798.00,gross,inpatient
2,FNA BIOPSY W/O IMA GDE 1ST LES,10021,NaN,IP Price,1370.00,gross,inpatient
3,IMG GUID DRN SOFT TIS CTH PERC,10030,NaN,IP Price,1654.00,gross,inpatient
4,ID SIMPLE ABSCESS,10060,NaN,IP Price,761.00,gross,inpatient
...,...,...,...,...,...,...,...
220439,HIV w major related condition w MCC,974,NaN,Intergroup,NaN,payer,
220440,Extensive O.R. procedure unrelated to principa...,981,NaN,Intergroup,NaN,payer,
220441,Extensive O.R. procedure unrelated to principa...,982,NaN,Intergroup,NaN,payer,
220442,Non-extensive O.R. proc unrelated to principal...,987,NaN,Intergroup,NaN,payer,


In [13]:
df_mid['billing_class'] = ''
df_mid['plan'] = ''
df_mid['local_code'] = ''
df_mid['rev_code'] = ''
df_mid['code'] = ''
df_mid['ms_drg'] = ''
df_mid['apr_drg'] = ''
df_mid['eapg'] = ''
df_mid['modifiers'] = ''

In [14]:
def filter_bad_ndc(ndc):
    components = ndc.split("-")
    if len(components) != 3:
        return 'na'
    
    l1 = len(components[0])
    l2 = len(components[1])
    l3 = len(components[2])
    
    if l1 == 4 and l2 == 4 and l3 == 2:
        return ndc
    elif l1 == 5 and l2 == 3 and l3 == 2:
        return ndc
    elif l1 == 5 and l2 == 4 and l3 == 1:
        return ndc
    elif l1 == 5 and l2 == 4 and l3 == 2:
        return ndc
    
    return 'na'

if 'ndc' in df_mid.columns.to_list():
    df_mid['ndc'] = df_mid['ndc'].fillna('')
    df_mid['ndc'] = df_mid['ndc'].apply(filter_bad_ndc)
else:
    df_mid['ndc'] = ''
    
df_mid

,description,code,ndc,payer,rate,payer_category,setting,billing_class,plan,local_code,rev_code,ms_drg,apr_drg,eapg,modifiers
0,FNA BX W/US GDN 1ST LES,,na,IP Price,798.00,gross,inpatient,,,,,,,,
1,FNA BX W/MR GDN EA ADDL,,na,IP Price,798.00,gross,inpatient,,,,,,,,
2,FNA BIOPSY W/O IMA GDE 1ST LES,,na,IP Price,1370.00,gross,inpatient,,,,,,,,
3,IMG GUID DRN SOFT TIS CTH PERC,,na,IP Price,1654.00,gross,inpatient,,,,,,,,
4,ID SIMPLE ABSCESS,,na,IP Price,761.00,gross,inpatient,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220439,HIV w major related condition w MCC,,na,Intergroup,NaN,payer,,,,,,,,,
220440,Extensive O.R. procedure unrelated to principa...,,na,Intergroup,NaN,payer,,,,,,,,,
220441,Extensive O.R. procedure unrelated to principa...,,na,Intergroup,NaN,payer,,,,,,,,,
220442,Non-extensive O.R. proc unrelated to principal...,,na,Intergroup,NaN,payer,,,,,,,,,


In [15]:
df_mid['hospital_id'] = ccn

In [16]:
def fix_drg(code):
    if len(code) == 1:
        return "00" + code
    if len(code) == 2 and code != "na":
        return "0" + code
    return code

df_mid['code'] = df_mid['code'].apply(fix_drg)

df_mid['hcpcs_cpt'] = df_mid.apply(lambda row: row['code'] if len(row['code']) == 5 else '', axis=1)
df_mid['ms_drg'] = df_mid.apply(lambda row: row['code'] if len(row['code']) == 3 else '', axis=1)

In [17]:
df_mid = df_mid[df_mid['rate'] != '-']

In [19]:
df_mid.columns

Index(['description', 'code', 'ndc', 'payer', 'rate', 'payer_category',
       'setting', 'billing_class', 'plan', 'local_code', 'rev_code', 'ms_drg',
       'apr_drg', 'eapg', 'modifiers', 'hospital_id', 'hcpcs_cpt'],
      dtype='object')

In [18]:
df_out = pd.DataFrame(columns=TARGET_COLUMNS)
df_out = pd.concat([df_out, df_mid])
df_out = df_out.dropna(subset=['rate'], axis=0)
df_out.to_csv(ccn + ".csv", index=False)
df_out

InvalidIndexError: Reindexing only valid with uniquely valued Index objects